In [ ]:
import gradio as gr
from PIL import Image
import torch
from torchvision import transforms, models
from groq import Groq
import json
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

# ----- 1. Load classifier model -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Recreate EfficientNet model architecture
model = models.efficientnet_b0(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_classes = 6
model.classifier = torch.nn.Sequential(
    torch.nn.Linear(model.classifier[1].in_features, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(256, num_classes)
)

# Load trained weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load("leather_defect_model.pth", map_location=device, weights_only=False)
model.eval()

# Classes
classes = ['Folding marks', 'Grain off', 'Growth marks', 'loose grains', 'non defective', 'pinhole']

# ----- 2. Define transforms -----
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

# ----- 3. Initialize Groq client -----
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY")) 

# ----- 4. Agentic prediction + explanation -----
def agentic_predict(img: Image.Image):
    # 1. Classifier prediction
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(img_tensor)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()[0]
        predicted_index = outputs.argmax(1).item()
        predicted_class = classes[predicted_index]
        confidence = float(probs[predicted_index])

    confidence_pct = round(confidence * 100, 1)

    # 2. Groq prompt
    prompt = f"""
You are an intelligent agent that handles leather image classification.

Given the following information:
- Predicted class: {predicted_class}
- Confidence score: {confidence_pct}%

Rules:
- Approve if the image is clearly non-defective.
- Reject if the image is clearly defective.
- Mark as 'Needs Review' if the model is uncertain.

Respond ONLY in strict JSON format like this:
{{
  "action": "Approve" | "Reject" | "Needs Review",
  "explanation": "2-3 sentence explanation"
}}
Do NOT include extra text outside JSON.
"""

    # 3. Call Groq chat completion API
    completion = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You explain decisions concisely."},
            {"role": "user", "content": prompt}
        ]
    )

    text = completion.choices[0].message.content

    # 4. Parse JSON robustly
    import re
    try:
        # Extract JSON substring from text
        match = re.search(r'\{.*\}', text, re.DOTALL)
        if match:
            output = json.loads(match.group())
            action = output.get("action", "Unknown")
            explanation = output.get("explanation", "")
        else:
            action = "Unknown"
            explanation = text.strip()
    except:
        action = "Unknown"
        explanation = text.strip()

    # 5. Return tuple for Gradio
    return (predicted_class, f"{confidence_pct}%", action, explanation)


# ----- 5. Gradio interface -----
interface = gr.Interface(
    fn=agentic_predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Textbox(label="Predicted Class", lines=2),
        gr.Textbox(label="Confidence", lines=2),
        gr.Textbox(label="Agent Action", lines=2),
        gr.Textbox(label="Explanation", lines=5)
    ],
    examples=None,
    live=False,
    title="Leather Defect Classifier with Agent Decisions",
    description="Upload a leather image and the model predicts the defect class, agent decides Approve/Reject/Needs Review, and explains its decision."
)

# ----- 6. Launch -----
interface.launch(share=True)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://de81e51900e858a3a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
